<a href="https://colab.research.google.com/github/iraidaantropova/StrD/blob/main/HW7_SinitsaI_StrD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Импорт библиотек
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

In [ ]:
#считываю данные
# Ниже,  для тех, у кого хоть раз были просрчоки больше 60 дней, ставлю в таргет 1.
data = pd.read_csv("application_record.csv", encoding = 'utf-8')
record = pd.read_csv("credit_record.csv", encoding = 'utf-8')

#  Добавляю срок кредита к параметрам выдачи кредита
begin_month = pd.DataFrame(record.groupby(["ID"])["MONTHS_BALANCE"].agg(min) * - 1)
begin_month = begin_month.rename(columns={'MONTHS_BALANCE':'begin_month'})
new_data = pd.merge(data, begin_month, how="left", on="ID")

#  Больше 60, то это просрочка, ставлю - Yes, если просрочка есть за срок кредита,то так же ставлю Yes
record['dep_value'] = None
record['dep_value'][record['STATUS'] == '2'] = 'Yes'
record['dep_value'][record['STATUS'] == '3'] = 'Yes'
record['dep_value'][record['STATUS'] == '4'] = 'Yes'
record['dep_value'][record['STATUS'] == '5'] = 'Yes'
cpunt = record.groupby('ID').count()
cpunt['dep_value'][cpunt['dep_value'] > 0] = 'Yes'
cpunt['dep_value'][cpunt['dep_value'] == 0] = 'No'

#  Джойним всё вместе,заменяю Yes и No на 1 и 0
cpunt = cpunt[['dep_value']]
new_data = pd.merge(new_data, cpunt, how='inner', on='ID')
new_data['target'] = new_data['dep_value']
new_data.loc[new_data['target'] == 'Yes', 'target'] = 1
new_data.loc[new_data['target'] == 'No', 'target'] = 0

In [ ]:
#  В итоге к анкетным данным добавляю таргет
new_data.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,...,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,begin_month,dep_value,target
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,-4542,1,1,0,0,NaN,2.0,15.0,No,0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,-4542,1,1,0,0,NaN,2.0,14.0,No,0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,...,-1134,1,0,0,0,Security staff,2.0,29.0,No,0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,...,-3051,1,0,1,1,Sales staff,1.0,4.0,No,0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,...,-3051,1,0,1,1,Sales staff,1.0,26.0,No,0


In [ ]:
# оставляю только часть признаков
features = ['AMT_INCOME_TOTAL', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN']
target = ['target',]
dataset = new_data[features + target]
dataset[target[0]] = pd.to_numeric(dataset[target[0]])

In [ ]:
# Разделяю выборку на трейн и тест, на трейн буду обучать модель, на тест валидировать.
X_train, X_test, y_train, y_test = train_test_split(dataset[features], pd.to_numeric(dataset[target[0]]), test_size=0.3, random_state=42)

In [ ]:
# Превращаю категориальные факторы в численные
ohe = OneHotEncoder()
ohe.fit(X_train[['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']])
X_train_ohe = ohe.transform(X_train[['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']])
X_test_ohe = ohe.transform(X_test[['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']])

X_train_ohe = pd.DataFrame(X_train_ohe.toarray(), columns=[item for sublist in ohe.categories_ for item in sublist])
X_test_ohe = pd.DataFrame(X_test_ohe.toarray(), columns=[item for sublist in ohe.categories_ for item in sublist])

In [ ]:
# Отскалирую численные
mms = MinMaxScaler()
mms.fit(X_train[['AMT_INCOME_TOTAL', 'CNT_CHILDREN']])
X_train_scaled = mms.transform(X_train[['AMT_INCOME_TOTAL', 'CNT_CHILDREN']])
X_test_scaled = mms.transform(X_test[['AMT_INCOME_TOTAL', 'CNT_CHILDREN']])

X_train_scaled = pd.DataFrame(X_train_scaled, columns=['AMT_INCOME_TOTAL', 'CNT_CHILDREN'])
X_test_scaled = pd.DataFrame(X_test_scaled, columns=['AMT_INCOME_TOTAL', 'CNT_CHILDREN'])

In [ ]:
X_train = pd.concat([X_train_scaled, X_train_ohe,], axis=1)
X_test = pd.concat([X_test_scaled, X_test_ohe, ], axis=1)

In [ ]:
# Создаю простейшую модель, которая покажет через линейные коэффиценты связь переменных и таргета
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
train_score, test_score = accuracy_score(model.predict(X_train), y_train), accuracy_score(model.predict(X_test), y_test)
print(f'Точность модели на трейне {train_score}, на тесте {test_score}')

Точность модели на трейне 0.9828755045260394, на тесте 0.983635033827025


In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285397 sha256=1f8b0b63a3165aa206d23f2121eae96f6e53dd8d6bf26f86f7d003365627ec8d
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
    .master("local[2]")\
    .appName("Lesson_7")\
    .config("spark.executor.instances",2)\
    .config("spark.executor.memory",'2g')\
    .config("spark.executor.cores",1)\
    .getOrCreate()
sc = spark.sparkContext

In [ ]:
spark

In [ ]:
from pyspark.ml import Estimator
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

from pyspark.ml.linalg import DenseVector
from pyspark.sql.types import *

import pyspark.sql.functions as F
from pyspark.ml.feature import HashingTF, Tokenizer, IDF
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.sql.functions import col, min, expr, when, count
from pyspark.sql.functions import desc, asc
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, MinMaxScaler
from pyspark.sql.types import IntegerType
from pyspark.ml.linalg import Vectors
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
#Считывание данных: для тех, у кого хоть раз были просрчоки больше 60 дней, ставлю в таргет 1.

data1 = spark.read.csv('application_record.csv', header=True, inferSchema=True)
record1 = spark.read.csv('credit_record.csv', header=True, inferSchema=True)

In [ ]:
#Добавляю срок кредита к параметрам выдачи кредита
begin_month = record1.groupBy("ID").agg(expr("-1 * min(MONTHS_BALANCE)").alias("begin_month"))
new_data1 = data1.join(begin_month, on='ID', how='left')

In [ ]:
#Больше 60, то это просрочка, ставлю - Yes, если просрочка есть за срок кредита,то так же ставлю Yes
# Джойним всё вместе,заменяем Yes и No на 1 и 0
record1 = record1.withColumn('dep_value', when((col("STATUS") == "2") | (col("STATUS") == '3') | (col("STATUS") == '4') | (col("STATUS") == '5'), "Yes"))
cpunt = record1.groupBy("ID").agg(*[count(c).alias(c + "_") for c in record1.columns])
cpunt = cpunt.withColumn('dep_value', when(col('dep_value_') > 0, 'Yes').
                                      when(col('dep_value_') == 0, 'No'))
cpunt2 = cpunt.select('ID', 'dep_value')
new_data1 = new_data1.join(cpunt2, on='ID', how='inner')
new_data1 = new_data1.withColumnRenamed('dep_value', 'target')
new_data1 = new_data1.withColumn('target', when(col('target') == 'Yes', 1).otherwise(0))

In [ ]:
#  В итоге к анкетным данным добавила таргет
new_data1.show(3)

+-------+-----------+------------+---------------+------------+----------------+----------------+--------------------+------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+-----------+------+
|     ID|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|NAME_INCOME_TYPE| NAME_EDUCATION_TYPE|NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|begin_month|target|
+-------+-----------+------------+---------------+------------+----------------+----------------+--------------------+------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+-----------+------+
|5008804|          M|           Y|              Y|           0|        427500.0|         Working|    Higher education|    Civil marriage| Rented apartment|  

In [ ]:
# оставляю только часть признаков, упрощая задачу
features = ['AMT_INCOME_TOTAL', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN']
target = ['target',]
dataset1 = new_data1.select(*(features + target))
# dataset1 = dataset1.withColumn("AMT_INCOME_TOTAL", col("AMT_INCOME_TOTAL").cast('vector'))
dataset1.explain

<bound method DataFrame.explain of DataFrame[AMT_INCOME_TOTAL: double, CODE_GENDER: string, FLAG_OWN_CAR: string, FLAG_OWN_REALTY: string, CNT_CHILDREN: int, target: int]>

In [ ]:
# Разделение датасета на train и test
train_ratio1 = 0.7
test_ratio1 = 1 - train_ratio1
train_data1, test_data1 = dataset1.randomSplit([train_ratio1, test_ratio1], seed=42)
train_data1.show()

+----------------+-----------+------------+---------------+------------+------+
|AMT_INCOME_TOTAL|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|target|
+----------------+-----------+------------+---------------+------------+------+
|         27000.0|          F|           N|              N|           0|     0|
|         27000.0|          F|           N|              N|           0|     0|
|         29250.0|          F|           Y|              Y|           0|     0|
|         29250.0|          F|           Y|              Y|           0|     0|
|         29250.0|          F|           Y|              Y|           0|     0|
|         29250.0|          F|           Y|              Y|           0|     0|
|         30150.0|          M|           N|              Y|           0|     0|
|         30150.0|          M|           N|              Y|           0|     0|
|         30150.0|          M|           N|              Y|           0|     0|
|         31500.0|          F|          

In [ ]:
categorical_cols = ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']
numeric_cols = ['AMT_INCOME_TOTAL', 'CNT_CHILDREN']


stages = []

for col in categorical_cols:
    indexer = StringIndexer(inputCol=col, outputCol=col + "_INDEX")
    stages.append(indexer)


pipeline = Pipeline(stages=stages)


indexed_train_data1 = pipeline.fit(train_data1).transform(train_data1)
indexed_test_data1 = pipeline.fit(test_data1).transform(test_data1)
# Show the transformed data
indexed_train_data1.show(3)
indexed_test_data1.show(3)

+----------------+-----------+------------+---------------+------------+------+-----------------+------------------+---------------------+
|AMT_INCOME_TOTAL|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|target|CODE_GENDER_INDEX|FLAG_OWN_CAR_INDEX|FLAG_OWN_REALTY_INDEX|
+----------------+-----------+------------+---------------+------------+------+-----------------+------------------+---------------------+
|         27000.0|          F|           N|              N|           0|     0|              0.0|               0.0|                  1.0|
|         27000.0|          F|           N|              N|           0|     0|              0.0|               0.0|                  1.0|
|         29250.0|          F|           Y|              Y|           0|     0|              0.0|               1.0|                  0.0|
+----------------+-----------+------------+---------------+------------+------+-----------------+------------------+---------------------+
only showing top 3 rows

+-

In [ ]:
# OneHotEncoder для категориальных колонок

onehot_encoder = OneHotEncoder(inputCols=[col + '_INDEX' for col in categorical_cols], outputCols=[col + "_encoded" for col in categorical_cols])

# Применяю OneHotEncoder к train_df
train_encoded1 = onehot_encoder.fit(indexed_train_data1).transform(indexed_train_data1)

# Применяю OneHotEncoder к test_df
test_encoded1 = onehot_encoder.fit(indexed_test_data1).transform(indexed_test_data1)

train_encoded1.show(3)
test_encoded1.show(3)

+----------------+-----------+------------+---------------+------------+------+-----------------+------------------+---------------------+-------------------+--------------------+-----------------------+
|AMT_INCOME_TOTAL|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|target|CODE_GENDER_INDEX|FLAG_OWN_CAR_INDEX|FLAG_OWN_REALTY_INDEX|CODE_GENDER_encoded|FLAG_OWN_CAR_encoded|FLAG_OWN_REALTY_encoded|
+----------------+-----------+------------+---------------+------------+------+-----------------+------------------+---------------------+-------------------+--------------------+-----------------------+
|         27000.0|          F|           N|              N|           0|     0|              0.0|               0.0|                  1.0|      (1,[0],[1.0])|       (1,[0],[1.0])|              (1,[],[])|
|         27000.0|          F|           N|              N|           0|     0|              0.0|               0.0|                  1.0|      (1,[0],[1.0])|       (1,[0],[1.0])|     

In [ ]:
# VectorAssembler для объединения признаков
assembler = VectorAssembler(inputCols=train_encoded1.columns[9:] + numeric_cols, outputCol="features")

# Применяем VectorAssembler к train_encoded
train_transformed1 = assembler.transform(train_encoded1).select("target", "features")

# Применяем VectorAssembler к test_encoded
test_transformed1 = assembler.transform(test_encoded1).select("target", "features")

In [ ]:
train_transformed1.show(3)

+------+--------------------+
|target|            features|
+------+--------------------+
|     0|[1.0,1.0,0.0,2700...|
|     0|[1.0,1.0,0.0,2700...|
|     0|[1.0,0.0,1.0,2925...|
+------+--------------------+
only showing top 3 rows



In [ ]:
# Создаю модель Logistic Regression
LR1 = LogisticRegression(featuresCol="features", labelCol="target")

# Обучаю модель на тренировочной выборке
model = LR1.fit(train_transformed1)

# Осуществляю прогнозирование на тестовой выборке
predictions = model.transform(test_transformed1)

# Провожу оценку модели
evaluator = BinaryClassificationEvaluator(labelCol="target")
accuracy = evaluator.evaluate(predictions)
print("Точность модели:", accuracy)

Точность модели: 0.5088000026189786


In [ ]:
spark.stop()